# Santander Product Recommendation

* The aim is to
* Modeling: Ensemble LightGBM, XGboost, NN(5 layers)

## 1) Prepare Data

* Load train and test data
* Assemble data together to preprocess

Import libraries and load train/test data

In [1]:
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np
import pickle

In [2]:
trn = pd.read_csv('../input/train_ver2.csv')
tst = pd.read_csv('../input/test_ver2.csv')

C:\Users\thisi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\thisi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
trn.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


Columns saving Customer varables/
Through EDA, we notice some data doesn't have any product data, which will be not helpful for building predicting model. Remove data of customers who doesn't own any products.

In [4]:
prods = trn.columns[24:]

trn[prods] = trn[prods].fillna(0.0).astype(np.int8)
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

Put train data and test data altogether for preprocessing data. As this project is from Kaggle, test data doesn't have product data. For now, we fill the data as 0 for preprocessing.<br>
After making all df, remove unnecessary data for making a space.

In [5]:
for col in trn.columns[24:]:
    tst[col] = 0
    
df = pd.concat([trn, tst], axis=0)
del trn, tst

df.shape

(12020685, 48)

## 2) Preprocessing Data

Removing unnecessary/duplicated data
* std 1:
* duplicated: cod_prov
* Of Customer variables, missing value is more than 90%, remove the data for prediction.

In [6]:
drop_ = ['nomprov', 'tipodom']

mask = (df.iloc[:,:24].isnull().sum() / df.shape[0]) > 0.9
drop_ += mask[mask].index.tolist()

df.drop(drop_, axis=1, inplace=True)
df.shape

(12020685, 44)

Feature list

In [7]:
features = []

### Preprocessing Categorical columns. 
<br>
1) 데이터 타입 맞추기:float to object, ind_nuevo, indrel, ind_actividad_cliente<br>
2) 이상치 수정하기 : 'indrel_1mes', ''tiprel_1mes'<br>

In [8]:
# Converting data type
flt_to_obj = ['ind_nuevo', 'indrel', 'ind_actividad_cliente']
df[flt_to_obj] = df[flt_to_obj].astype('O')

# Remove noise
df['indrel_1mes'].replace({'1.0':1, '1':1, 2.0:2, '2.0':2,'2':2, 3.0:3, '3':3, '3.0': 3, '4':4, 4.0:4,'4.0': 4, 'P':5}, inplace=True)
df['tiprel_1mes'].replace('N', 'I', inplace=True)

In [9]:
categorical_cols = ['ind_empleado', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'indfall', 'segmento', 'indrel_1mes'] + flt_to_obj + ['pais_residencia', 'canal_entrada','cod_prov']

df[categorical_cols] = df[categorical_cols].fillna(-99)

for col in categorical_cols:
    df[col], _ = df[col].factorize()
    
features += categorical_cols

## Numerical columns
* age: Age
* antiguedad: Customer seniority (in months)
* renta : Gross income of the household

In [10]:
#데이터 타입 숫자로 수정
def str_to_int(series):    
    if type(series) is str:
        return int(series.strip())
    else:
        return series

#수치형 변수 1:age
df['age'].replace(' NA',-99, inplace=True)
df['age'] = df['age'].apply(str_to_int).astype(np.int8)

#수치형 변수 2:antiguedad
df['antiguedad'].replace(['     NA','-999999',-999999], -99, inplace=True)
df['antiguedad'] = df['antiguedad'].apply(str_to_int).astype(np.int8)

#수치형 변수3: renta
df['renta'].fillna(-99, inplace=True)
df['renta'].replace('         NA', -99, inplace=True)
df['renta'] = df['renta'].apply(float).round()

# 학습에 사용할 수치형 변수를 features에 추구한다.
features += ['age', 'antiguedad' ,'renta']

# Feature Engineering

To identify newly added products, make lag-1 product data
* Step 1) convert date to int to see order

In [11]:
# 날짜 변수에서 연도와 월 정보를 추출한다.
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)

In [12]:
# 날짜를 숫자로 변환하는 함수이다. 2015-01-28은 1, 2016-06-28은 18로 변환된다
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [13]:
# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col+'_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. Lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

# 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다
del df, df_lag

In [14]:
# 저번 달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
df_trn.fillna(0, inplace=True)

In [15]:
# lag-1 변수를 추가한다.
features += [f+'_prev' for f in features]
features += [p+'_prev' for p in prods]

In [16]:
features += ['fecha_alta_month', 'fecha_alta_year']

# Test and Data Split
<br>
First, we are going to validate data with train data and also test data.<br>

In [17]:
vld_date = '2016-05-28'
tst_date = '2016-06-28'

tst_vld = df_trn[df_trn['fecha_dato']== vld_date]
tst_all = df_trn[df_trn['fecha_dato'] == tst_date]

trn = df_trn[df_trn['fecha_dato']< tst_date]

#del df_trn

For validation process, we extract data of customer who actually buy new products

In [18]:
# 훈련 데이터에서 신규 구매 건수만 추출한다.
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)

XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

Recently bought products

In [22]:
target = sorted(XY[XY['fecha_dato']>'2015-05-28']['y'].value_counts().index.tolist()[:16])
non_target = sorted(XY[XY['fecha_dato']>'2015-05-28']['y'].value_counts().index.tolist()[16:])
target, non_target

([2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 17, 18, 19, 21, 22, 23],
 [0, 1, 3, 10, 14, 15, 16, 20])

If there was no addition, make it 

In [19]:
prevs = [prod+'_prev' for prod in prods]

arr = trn[prods].values - trn[prevs].values
arr = np.sum(arr, axis=1)
non_index = []
for i in range(len(arr)):
    if arr[i]==0:
        non_index.append(i)
        


In [22]:
trn.shape[0], XY.shape[0]len(non_index)

(11091070, 1778875, 9676014)

In [ ]:
trn_non = trn.iloc[non_index]
trn_non['y'] = 24
trn_non['target'] = 16

In [24]:
tarX=[]
for i,t in enumerate(target):
    tX = XY[XY['y'] == t]
    tX['target'] = i
    tarX.append(tX)

for nt in non_target:
    tX = XY[XY['y'] == nt]
    tX['target'] = 16
    tarX.append(tX)

tarX.append(trn_non)
del trn_non, XY

trn_17 = pd.concat(tarX)

C:\Users\thisi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\thisi\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [30]:
with open('../input/meta_data.pkl', 'wb') as fout:
    pickle.dump({
        'features':features,
        'prods':prods,
        'target':target
    }, fout)

In [34]:
with open('../input/processed_data.pkl', 'wb') as foutt:
    pickle.dump({
        'trn_all': trn_17,
        'tst_vld': tst_vld,
        'tst_all': tst_all
    }, foutt)